# Large Scale Data Mining 1st assignment
Francisco Marques 97639

#### Initialize Spark session and load file as RDD

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

conf = SparkConf()
sc = SparkContext(conf=conf)

In [14]:
support_threshold = 1000
k = [2, 3]
inputFile = 'conditions.csv.gz'
file = sc.textFile(inputFile).sample(False, 0.1) # load file as RDD

Map the RDD to split the lines and put all of the items in just one array

It only keeps the columns PATIENT, CODE and DESCRIPTION, so k = 1,2,3.

In [15]:
# lines in one array
wlines = (file.flatMap(lambda line: line.strip().split(','))
        .filter(lambda x: x != '')
        ) # filters out blank elements
#wlines = wlines.flatMap(lambda line: [line[2], line[-2], line[-1]])
wlines.take(10)

['2015-01-04',
 '2015-01-18',
 'b0a03e8c-8d0f-4242-9548-40f4d294eba8',
 'fb838ab5-2805-41c0-bd6f-55340284c98e',
 '10509002',
 'Acute bronchitis (disorder)',
 '2015-01-28',
 'bf1f30f2-27de-4b54-809b-f91b92949565',
 '99d8e5ca-56f4-40e7-a4ef-f1f6db881553',
 '239873007']

Create new RDD with unique items. Create a new support RDD that has the pair (item, 1) that will be used to count the occurrences of each item.

In [31]:
uniqueItems = wlines.distinct()

supportRdd = wlines.map(lambda x: (x, 1))

Reduce by adding the '1s' in each pair, resulting in the counting of every item.

In [32]:
supportRdd = supportRdd.reduceByKey(lambda x, y: x + y)
supportRdd.take(10)

[('2015-01-04', 220),
 ('2015-01-18', 217),
 ('b0a03e8c-8d0f-4242-9548-40f4d294eba8', 1),
 ('fb838ab5-2805-41c0-bd6f-55340284c98e', 1),
 ('10509002', 57938),
 ('Acute bronchitis (disorder)', 57938),
 ('2015-01-28', 259),
 ('bf1f30f2-27de-4b54-809b-f91b92949565', 1),
 ('99d8e5ca-56f4-40e7-a4ef-f1f6db881553', 1),
 ('239873007', 6042)]

Create a new RDD with the most frequent items, that exceed the support threshold (1000).

In [33]:
frequentRdd = supportRdd.filter(lambda item: item[1] >= support_threshold)

supportRdd = frequentRdd.map(lambda x: x[0]) # makes the support rdd only contain keys again

frequentRdd.take(10)

[('10509002', 57938),
 ('Acute bronchitis (disorder)', 57938),
 ('239873007', 6042),
 ('Osteoarthritis of knee', 6042),
 ('72892002', 50456),
 ('Normal pregnancy', 50456),
 ('429007001', 4421),
 ('History of cardiac arrest (situation)', 4421),
 ('55822004', 13406),
 ('Hyperlipidemia', 13406)]

10 most frequent items of this first RDD.

In [34]:
frequentRdd.top(10, key = lambda x: x[1])

[('444814009', 123135),
 ('Viral sinusitis (disorder)', 123135),
 ('195662009', 70355),
 ('Acute viral pharyngitis (disorder)', 70355),
 ('10509002', 57938),
 ('Acute bronchitis (disorder)', 57938),
 ('72892002', 50456),
 ('Normal pregnancy', 50456),
 ('162864005', 36401),
 ('Body mass index 30+ - obesity (finding)', 36401)]

The first pass is now done, so next it will be counting the frequent items for k = 2.

First we create an RDD with all the combinations of the most frequent for k=1 with every unique value.

In [35]:
combined = (supportRdd.cartesian(uniqueItems)
            .filter(lambda item: item[0] != item[1])
            .map(lambda item: (item, 1))
            )

combined.take(10)

[(('10509002', '2015-01-04'), 1),
 (('10509002', '2015-01-18'), 1),
 (('10509002', 'b0a03e8c-8d0f-4242-9548-40f4d294eba8'), 1),
 (('10509002', 'fb838ab5-2805-41c0-bd6f-55340284c98e'), 1),
 (('10509002', 'Acute bronchitis (disorder)'), 1),
 (('10509002', '2015-01-28'), 1),
 (('10509002', 'bf1f30f2-27de-4b54-809b-f91b92949565'), 1),
 (('10509002', '99d8e5ca-56f4-40e7-a4ef-f1f6db881553'), 1),
 (('10509002', '239873007'), 1),
 (('10509002', 'Osteoarthritis of knee'), 1)]

In [37]:
combinedSupport = combined.reduceByKey(lambda x, y: x[1] + y[1])
combinedSupport.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 71.0 failed 1 times, most recent failure: Lost task 0.0 in stage 71.0 (TID 37) (192.168.42.98 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:592)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:574)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:763)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:740)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:505)
	at org.apache.spark.InterruptibleIterator.hasNext(Interrupt

In [29]:
combinedFrequent = combinedSupport.filter(lambda item: item[1] >= support_threshold)

combinedSupport = combinedFrequent.map(lambda x: x[0]) # makes the support rdd only contain keys again

combinedFrequent.take(10)

[]

Function to remove replicas such as $(a,b)$ and $(b,a)$

In [9]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [11]:
combined = combined.map(lambda item: removeReplica(item))
combined = combined.filter(lambda item: len(item) == 2)
combined.take(10)

[('444814009', 'START'),
 ('444814009', 'STOP'),
 ('444814009', 'PATIENT'),
 ('444814009', 'ENCOUNTER'),
 ('444814009', 'CODE'),
 ('444814009', 'DESCRIPTION'),
 ('2019-04-27', '444814009'),
 ('2019-05-18', '444814009'),
 ('09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a', '444814009'),
 ('411d4eae-72d1-4786-a28e-3b744cf17a47', '444814009')]